In [1]:
import numpy as np
import pandas as pd

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    f1_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef,
    classification_report, confusion_matrix, log_loss
)
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime

In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [3]:
rows_fin2 = session.execute("""
    SELECT symbol,eps,pe,pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fundamental= pd.DataFrame(rows_fin2)
df_fundamental.head(10)

,symbol,eps,pe,pbv,percentyield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00
5,VCOM,0.21428,7.85,1.48,10.79
6,KDH,4.12186,10.62,1.93,3.01
7,NVD,-0.00942,38.87,0.32,0.00
8,JDF,0.04224,12.19,1.35,4.85
9,SVR,-0.03402,0.00,0.34,0.00


In [4]:
rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_candlestick_latest = pd.DataFrame(rows_price)
df_candlestick_latest['time'] = pd.to_datetime(df_candlestick_latest['time'])
df_candlestick_latest = df_candlestick_latest.sort_values(['symbol', 'time'])

In [5]:
# merge fundamental & candlestick
df = pd.merge(df_fundamental, df_candlestick_latest, on="symbol")


# คำนวณ features
df["marketcap"] = df["close_price"] * df["volume"]

In [6]:
df.head()

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap
0,PPPM,0.02,0.0,0.7,0.0,2022-06-13,2.31114,2.66670,2.31114,2.66670,0.0,18958630,4.381605e+07
1,PPPM,0.02,0.0,0.7,0.0,2022-06-14,2.13336,2.48892,2.13336,2.31114,0.0,36479858,7.782467e+07
2,PPPM,0.02,0.0,0.7,0.0,2022-06-15,2.13336,2.31114,2.13336,2.13336,0.0,35826032,7.642982e+07
3,PPPM,0.02,0.0,0.7,0.0,2022-06-16,2.31114,2.48892,2.13336,2.31114,0.0,28035706,6.479444e+07
4,PPPM,0.02,0.0,0.7,0.0,2022-06-17,2.31114,2.48892,2.13336,2.31114,0.0,21671802,5.008657e+07


In [10]:
for span in [5, 15, 35, 89, 200]:
    df[f'ema{span}'] = df.groupby('symbol')['close_price'].transform(lambda x: x.ewm(span=span, adjust=False).mean())

# ✅ คำนวณ RSI
import ta

df['rsi'] = df.groupby('symbol')['close_price'].transform(
    lambda x: ta.momentum.RSIIndicator(close=x, window=14).rsi()
)

# ✅ ฟังก์ชันแบ่ง trend_group ตามเงื่อนไข
import numpy as np

def trend_group(row):
    try:
        c = row['close_price']
        r = row['rsi']
        e5, e15, e35, e89, e200 = row['ema5'], row['ema15'], row['ema35'], row['ema89'], row['ema200']

        if c >= e5 and r >= 70:
            return 'a'  # Overbought
        elif c >= e35 and e35 >= e89:
            return 'b'  # Uptrend
        elif c >= e89 and (max([e5,e15,e35,e89]) - min([e5,e15,e35,e89])) / np.mean([e5,e15,e35,e89]) <= 0.015:
            return 'c'  # Sideways
        elif c < e89 and c < e200 and e89 < e200:
            return 'd'  # Downtrend
        elif c < e5 < e15 < e35 < e89 < e200 and r <= 30:
            return 'e'  # Crash
        else:
            return None
    except:
        return None

# ✅ apply trend_group
df['trend_group'] = df.apply(trend_group, axis=1)

In [11]:
df.head(10)

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap,ema5,ema15,ema35,ema89,ema200,rsi,trend_group
0,PPPM,0.02,0.0,0.7,0.0,2022-06-13,2.31114,2.66670,2.31114,2.66670,0.0,18958630,4.381605e+07,2.311140,2.311140,2.311140,2.311140,2.311140,NaN,b
1,PPPM,0.02,0.0,0.7,0.0,2022-06-14,2.13336,2.48892,2.13336,2.31114,0.0,36479858,7.782467e+07,2.251880,2.288918,2.301263,2.307189,2.309371,NaN,d
2,PPPM,0.02,0.0,0.7,0.0,2022-06-15,2.13336,2.31114,2.13336,2.13336,0.0,35826032,7.642982e+07,2.212373,2.269473,2.291935,2.303327,2.307620,NaN,d
3,PPPM,0.02,0.0,0.7,0.0,2022-06-16,2.31114,2.48892,2.13336,2.31114,0.0,28035706,6.479444e+07,2.245296,2.274681,2.293002,2.303500,2.307655,NaN,None
4,PPPM,0.02,0.0,0.7,0.0,2022-06-17,2.31114,2.48892,2.13336,2.31114,0.0,21671802,5.008657e+07,2.267244,2.279239,2.294010,2.303670,2.307689,NaN,None
5,PPPM,0.02,0.0,0.7,0.0,2022-06-20,2.13336,2.31114,2.13336,2.31114,0.0,1648029,3.515839e+06,2.222616,2.261004,2.285085,2.299885,2.305955,NaN,d
6,PPPM,0.02,0.0,0.7,0.0,2022-06-21,2.31114,2.48892,2.13336,2.31114,0.0,35683156,8.246877e+07,2.252124,2.267271,2.286532,2.300135,2.306006,NaN,None
7,PPPM,0.02,0.0,0.7,0.0,2022-06-22,2.66670,2.84448,2.31114,2.48892,0.0,92032824,2.454239e+08,2.390316,2.317199,2.307653,2.308281,2.309595,NaN,None
8,PPPM,0.02,0.0,0.7,0.0,2022-06-23,2.48892,2.84448,2.48892,2.84448,0.0,26127032,6.502809e+07,2.423184,2.338664,2.317723,2.312295,2.311380,NaN,b
9,PPPM,0.02,0.0,0.7,0.0,2022-06-24,2.48892,2.66670,2.31114,2.66670,0.0,25062537,6.237865e+07,2.445096,2.357446,2.327234,2.316220,2.313146,NaN,b


In [13]:
# ✅ Clustering กลุ่ม A–E ด้วย KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = ['eps', 'pe', 'pbv', 'percentyield', 'marketcap']
X = df[features].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
kmeans_labels = kmeans.fit_predict(X_scaled)
df.loc[X.index, 'group'] = kmeans_labels

group_map = {i: chr(65+i) for i in range(5)}
df['group'] = df['group'].map(group_map)

In [14]:
# ✅ รวม label → quadrant
mask = df['group'].notna() & df['trend_group'].notna()
df['quadrant'] = None
df.loc[mask, 'quadrant'] = df.loc[mask, 'group'] + df.loc[mask, 'trend_group']

In [15]:
# ✅ สร้าง Deep Learning Classifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

label_encoder = LabelEncoder()
df_model = df[mask].copy()
y = label_encoder.fit_transform(df_model['quadrant'])
X = df_model[features + ['rsi', 'ema5', 'ema15', 'ema35', 'ema89', 'ema200']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.4768 - loss: 258233.5469 - val_accuracy: 0.7525 - val_loss: 0.9308
Epoch 2/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7528 - loss: 2.3648 - val_accuracy: 0.7528 - val_loss: 0.8852
Epoch 3/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.7529 - loss: 7.2171 - val_accuracy: 0.7526 - val_loss: 0.8857
Epoch 4/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.7519 - loss: 4.4236 - val_accuracy: 0.7525 - val_loss: 0.8922
Epoch 5/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.7532 - loss: 4.5864 - val_accuracy: 0.7525 - val_loss: 0.8859
Epoch 6/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7513 - loss: 13.1639 - val_accuracy: 0.7525 - val_loss: 0.8930
Epoch 7/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7525 - loss: 8.2147 - val_accuracy: 0.7524 - val_loss: 0.8873
Epoch 8/50
11646/11646 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accu

In [18]:
# ✅ Evaluate Accuracy
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")


✅ Model Accuracy: 75.24%


In [17]:
df_heat.head()

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,...,marketcap,ema5,ema15,ema35,ema89,ema200,rsi,trend_group,group,quadrant
0,PPPM,0.02,0.0,0.7,0.0,2022-06-13,2.31114,2.66670,2.31114,2.66670,...,4.381605e+07,2.311140,2.311140,2.311140,2.311140,2.311140,NaN,b,A,Ab
1,PPPM,0.02,0.0,0.7,0.0,2022-06-14,2.13336,2.48892,2.13336,2.31114,...,7.782467e+07,2.251880,2.288918,2.301263,2.307189,2.309371,NaN,d,A,Ad
2,PPPM,0.02,0.0,0.7,0.0,2022-06-15,2.13336,2.31114,2.13336,2.13336,...,7.642982e+07,2.212373,2.269473,2.291935,2.303327,2.307620,NaN,d,A,Ad
5,PPPM,0.02,0.0,0.7,0.0,2022-06-20,2.13336,2.31114,2.13336,2.31114,...,3.515839e+06,2.222616,2.261004,2.285085,2.299885,2.305955,NaN,d,A,Ad
8,PPPM,0.02,0.0,0.7,0.0,2022-06-23,2.48892,2.84448,2.48892,2.84448,...,6.502809e+07,2.423184,2.338664,2.317723,2.312295,2.311380,NaN,b,A,Ab


In [16]:
import plotly.graph_objects as go

df_heat = df.dropna(subset=['quadrant'])
quadrant_map = df_heat.groupby("quadrant")["symbol"].apply(list).to_dict()

x_labels = ['a', 'b', 'c', 'd', 'e']
y_labels = ['A', 'B', 'C', 'D', 'E']
z_matrix = []
text_matrix = []

for y in y_labels:
    row_z, row_text = [], []
    for x in x_labels:
        key = y + x
        stocks = quadrant_map.get(key, [])
        row_z.append(len(stocks))
        row_text.append(f"{key}:<br>" + "<br>".join(stocks) if stocks else f"{key}:<br>No stocks")
    z_matrix.append(row_z)
    text_matrix.append(row_text)

fig = go.Figure(data=go.Heatmap(z=z_matrix, x=x_labels, y=y_labels, text=text_matrix, hoverinfo='text'))
fig.update_layout(title='📊 Stock Quadrant Heatmap', xaxis_title='Trend (a–e)', yaxis_title='Financial (A–E)')
fig.show()